In [12]:
import tensorflow as tf
import numpy as np
import pickle

with open('../data/conv_lstm_sequence_data.pc', 'rb') as f:
    raw_data = pickle.load(f)
raw_data = np.expand_dims(raw_data, axis=-1)
print("raw dataset shape: {}".format(raw_data.shape))

raw dataset shape: (17006, 30, 40, 1)


In [13]:
### THESE CODES ARE MEANT FOR TF2

In [14]:
train, test = raw_data[:15000], raw_data[15000:]
print("train, test shapes: {}-{}".format(train.shape, test.shape))

train, test shapes: (15000, 30, 40, 1)-(2006, 30, 40, 1)


In [15]:
from tensorflow.data import Dataset

seq_len = 18 # it's 1 hour and a half
batch_size = 10

train_ds = Dataset.from_tensor_slices(train)
test_ds = Dataset.from_tensor_slices(test)

test_seq = train_ds.batch(seq_len+1, drop_remainder=True)
sequences = train_ds.batch(seq_len+1, drop_remainder=True)

test_ds = test_seq.map(lambda x: (x[:-1], x[1:]))
test_ds = test_ds.batch(batch_size, drop_remainder=True)
ds = sequences.map(lambda x: (x[:-1], x[1:]))
ds = ds.shuffle(10000).batch(batch_size, drop_remainder=True)

In [16]:
ds

<BatchDataset shapes: ((10, 18, 30, 40, 1), (10, 18, 30, 40, 1)), types: (tf.float64, tf.float64)>

In [19]:
from tensorflow.keras.models import Sequential

model = Sequential([
    tf.keras.layers.ConvLSTM2D(filters=20, kernel_size=(3,3), padding='same', input_shape=(None, 30, 40, 1), return_sequences=True),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ConvLSTM2D(filters=20, kernel_size=(3,3), padding='same', return_sequences=True),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv3D(filters=1, kernel_size=(3,3,3), activation='sigmoid', padding='same')
])

model.compile(loss='mse', optimizer='adam')
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_6 (ConvLSTM2D)  (None, None, 30, 40, 20)  15200     
_________________________________________________________________
batch_normalization_6 (Batch (None, None, 30, 40, 20)  80        
_________________________________________________________________
conv_lst_m2d_7 (ConvLSTM2D)  (None, None, 30, 40, 20)  28880     
_________________________________________________________________
batch_normalization_7 (Batch (None, None, 30, 40, 20)  80        
_________________________________________________________________
conv3d_3 (Conv3D)            (None, None, 30, 40, 1)   541       
Total params: 44,781
Trainable params: 44,701
Non-trainable params: 80
_________________________________________________________________


In [20]:
model.fit(ds, epochs=5)

Epoch 1/5
78/78 [==============================] - 182s 2s/step - loss: 0.3850
Epoch 2/5
78/78 [==============================] - 182s 2s/step - loss: 0.3198
Epoch 3/5
78/78 [==============================] - 177s 2s/step - loss: 0.3028
Epoch 4/5
78/78 [==============================] - 178s 2s/step - loss: 0.2990
Epoch 5/5
78/78 [==============================] - 173s 2s/step - loss: 0.2981


In [21]:
model.evaluate(test_ds)

78/78 [==============================] - 48s 614ms/step - loss: 0.3441


0.34407910589988416

In [23]:
for x_batch, y_batch in test_ds.take(10):
    for x_test, y_test in 

(10, 18, 30, 40, 1) (10, 18, 30, 40, 1)
(10, 18, 30, 40, 1) (10, 18, 30, 40, 1)
(10, 18, 30, 40, 1) (10, 18, 30, 40, 1)
(10, 18, 30, 40, 1) (10, 18, 30, 40, 1)
(10, 18, 30, 40, 1) (10, 18, 30, 40, 1)
(10, 18, 30, 40, 1) (10, 18, 30, 40, 1)
(10, 18, 30, 40, 1) (10, 18, 30, 40, 1)
(10, 18, 30, 40, 1) (10, 18, 30, 40, 1)
(10, 18, 30, 40, 1) (10, 18, 30, 40, 1)
(10, 18, 30, 40, 1) (10, 18, 30, 40, 1)


In [5]:
### THESE CODES ARE MEANT FOR TF1.14

In [18]:
from keras.models import Sequential
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization
import numpy as np
import pylab as plt

# We create a layer which take as input movies of shape
# (n_frames, width, height, channels) and returns a movie
# of identical shape.

with tf.device('/cpu:0'):
    model = Sequential()
    model.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                       input_shape=(None, 30, 40, 1),
                       padding='same', return_sequences=True))
    model.add(BatchNormalization())

    model.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                       padding='same', return_sequences=True))
    model.add(BatchNormalization())

    model.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                       padding='same', return_sequences=True))
    model.add(BatchNormalization())

    model.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                       padding='same', return_sequences=True))
    model.add(BatchNormalization())

    model.add(Conv3D(filters=1, kernel_size=(3, 3, 3),
                   activation='sigmoid',
                   padding='same', data_format='channels_last'))
    model.compile(loss='binary_crossentropy', optimizer='adadelta')

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_17 (ConvLSTM2D) (None, None, 30, 40, 40)  59200     
_________________________________________________________________
batch_normalization_17 (Batc (None, None, 30, 40, 40)  160       
_________________________________________________________________
conv_lst_m2d_18 (ConvLSTM2D) (None, None, 30, 40, 40)  115360    
_________________________________________________________________
batch_normalization_18 (Batc (None, None, 30, 40, 40)  160       
_________________________________________________________________
conv_lst_m2d_19 (ConvLSTM2D) (None, None, 30, 40, 40)  115360    
_________________________________________________________________
batch_normalization_19 (Batc (None, None, 30, 40, 40)  160       
_________________________________________________________________
conv_lst_m2d_20 (ConvLSTM2D) (None, None, 30, 40, 40)  115360    
__________

In [12]:
print(raw_data.shape)
seq_len = 18
bs = 10

remainder = raw_data.shape[0]%seq_len
raw_data = raw_data[:raw_data.shape[0]-remainder,:]

print(raw_data.shape)

seq = np.array(np.split(raw_data, raw_data.shape[0]//seq_len))

(17006, 30, 40, 1)
(16992, 30, 40, 1)


In [13]:
input_data, target = seq[:,:-1, :], seq[:, 1:, :]
print(input_data.shape, target.shape)


(944, 17, 30, 40, 1) (944, 17, 30, 40, 1)


In [ ]:
model.fit(input_data, target, batch_size=10, epochs=10, validation_split=0.05)

Train on 896 samples, validate on 48 samples
Epoch 1/10
700/896 [======================>.......] - ETA: 2:50 - loss: -0.7902

In [31]:
from tensorflow.data import Dataset

seq_len = 18 # it's 1 hour and a half
batch_size = 5

ds = Dataset.from_tensor_slices(raw_data)
sequences = ds.batch(seq_len+1, drop_remainder=True).shuffle(10000).batch(batch_size, drop_remainder=True)

ds = sequences.map(lambda x: x[:,:-1])
target = sequences.map(lambda x: x[:,1:])


In [33]:
ds, target

(<DatasetV1Adapter shapes: (5, 18, 30, 40, 1), types: tf.float64>,
 <DatasetV1Adapter shapes: (5, 18, 30, 40, 1), types: tf.float64>)

In [36]:
seq.fit(ds, target, steps_per_epoch=10)

AttributeError: 'DatasetV1Adapter' object has no attribute 'ndim'